In [1]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import shutil


In [2]:
def clear_directory(directory):
    # Mengecek apakah direktori ada dan tidak kosong
    if os.path.exists(directory) and len(os.listdir(directory)) > 0:
        # Menghapus semua isi direktori
        for file in os.listdir(directory):
            file_path = os.path.join(directory, file)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print('Gagal menghapus %s. Alasan: %s' % (file_path, e))
        print(f"Isi folder {directory} telah dihapus.")
    else:
        print(f"Folder {directory} sudah kosong atau tidak ada.")

# Lokasi direktori dasar
base_dir = 'D:/VCS_Project-main/TA_Unet/source/data'

# Membuat folder jika belum ada
os.makedirs(os.path.join(base_dir, 'raw_patches'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'mask_patches'), exist_ok=True)

# Mengecek dan mengosongkan folder jika perlu
clear_directory(os.path.join(base_dir, 'raw_patches'))
clear_directory(os.path.join(base_dir, 'mask_patches'))


Isi folder D:/VCS_Project-main/TA_Unet/source/data\raw_patches telah dihapus.
Isi folder D:/VCS_Project-main/TA_Unet/source/data\mask_patches telah dihapus.


In [3]:
def crop_image(img_path, output_folder, prefix, size=128):
    img = Image.open(img_path)
    img_width, img_height = img.size
    crops = []

    # Membuat potongan gambar
    for i in range(0, img_width, size):
        for j in range(0, img_height, size):
            # Memastikan potongan tidak melebihi batas gambar
            box = (i, j, i + size if i + size <= img_width else img_width, j + size if j + size <= img_height else img_height)
            img_crop = img.crop(box)
            crop_path = os.path.join(output_folder, f"{prefix}_{i}_{j}.tif")
            img_crop.save(crop_path)
            crops.append(crop_path)
    return crops


In [4]:
def process_images(csv_path, raw_folder, mask_folder):
    df = pd.read_csv(csv_path)
    new_rows = []

    # Memproses setiap baris dalam DataFrame
    for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing images"):
        sat_crops = crop_image(row['sat_image_path'], raw_folder, row['image_id'] + '_sat')
        mask_crops = crop_image(row['mask_path'], mask_folder, row['image_id'] + '_mask')

        for sat_path, mask_path in zip(sat_crops, mask_crops):
            new_rows.append({
                'image_id': row['image_id'],
                'sat_image_path': sat_path,
                'mask_path': mask_path,
                'split': row['split']
            })

    # Simpan ke CSV baru
    new_df = pd.DataFrame(new_rows)
    new_df.to_csv(os.path.join(os.path.dirname(csv_path), 'new_dataset.csv'), index=False)

# Jalankan fungsi pemrosesan
csv_file_path = os.path.join(base_dir, 'metadata.csv')  # Lokasi metadata.csv yang disesuaikan
raw_patches_folder = os.path.join(base_dir, 'raw_patches')
mask_patches_folder = os.path.join(base_dir, 'mask_patches')
process_images(csv_file_path, raw_patches_folder, mask_patches_folder)


Processing images:   0%|          | 0/34 [00:00<?, ?it/s]

Processing images: 100%|██████████| 34/34 [04:52<00:00,  8.60s/it]
